# Ayurvedic Taste Profile Data Collection and Model

This notebook provides:
1. Data structure for collecting Ayurvedic taste profiles
2. Input validation for taste ratings
3. Standardized rating scales
4. Interactive data collection form
5. Data export functionality
6. Sample data generation

## Required Libraries

In [9]:
import pandas as pd
import numpy as np
from datetime import datetime
import ipywidgets as widgets
from IPython.display import display, clear_output
import json
from pathlib import Path
import joblib
from sklearn.ensemble import RandomForestRegressor

## Data Structure Setup

Define the data structure for collecting Ayurvedic taste profiles and sensor readings:

In [ ]:
# Define column names
SENSOR_COLUMNS = [
    'alcohol_ppm',  # New alcohol sensor reading
    'as7263_r',
    'as7263_s', 
    'as7263_t',
    'as7263_u',
    'as7263_v',
    'as7263_w'
]

TASTE_COLUMNS = [
    'sweet',     # Madhura
    'sour',      # Amla
    'salty',     # Lavana
    'bitter',    # Tikta
    'pungent',   # Katu
    'astringent' # Kashaya
]

# Create empty DataFrame
columns = [
    'sample_id',
    'timestamp',
    'factory_name',
    'medicine_name',
    *SENSOR_COLUMNS,
    *[f'taste_{taste}' for taste in TASTE_COLUMNS],
    'quality',
    'dilution'
]

df = pd.DataFrame(columns=columns)
print("Data structure created with columns:", columns)

Data structure created with columns: ['sample_id', 'timestamp', 'medicine_name', 'dilution', 'temperature', 'as7263_r', 'as7263_s', 'as7263_t', 'as7263_u', 'as7263_v', 'as7263_w', 'sweet', 'sour', 'salty', 'bitter', 'pungent', 'astringent', 'effectiveness']


In [ ]:
# Weighted averaging function for sensor readings
def weighted_average_readings(readings_list, num_samples=10):
    """
    Calculate weighted average of sensor readings with increasing weights.
    Last reading has highest weight (1.0).
    
    Args:
        readings_list: List of dictionaries containing sensor readings
        num_samples: Number of samples to consider (default=10)
    
    Returns:
        Dictionary with weighted average values for each sensor
    """
    if len(readings_list) < num_samples:
        raise ValueError(f"Need at least {num_samples} readings. Got {len(readings_list)}")
    
    # Use last num_samples readings
    readings = readings_list[-num_samples:]
    
    # Generate weights with increasing values
    weights = np.linspace(0.1, 1.0, num_samples)
    weights = weights / np.sum(weights)  # Normalize weights
    
    # Initialize result dictionary
    result = {}
    
    # Calculate weighted average for each sensor
    for column in SENSOR_COLUMNS:
        values = [reading[column] for reading in readings]
        result[column] = np.average(values, weights=weights)
    
    return result

# Test weighted averaging function
def generate_test_readings(num_samples=10):
    """Generate test readings with some noise."""
    readings = []
    base_values = {
        'alcohol_ppm': 50,
        'as7263_r': 0.5,
        'as7263_s': 0.4,
        'as7263_t': 0.6,
        'as7263_u': 0.3,
        'as7263_v': 0.7,
        'as7263_w': 0.5
    }
    
    for _ in range(num_samples):
        reading = {
            key: value + np.random.normal(0, 0.05) 
            for key, value in base_values.items()
        }
        readings.append(reading)
    
    return readings

# Test the weighted averaging
test_readings = generate_test_readings()
weighted_result = weighted_average_readings(test_readings)

print("\nTest weighted averaging function:")
for sensor, value in weighted_result.items():
    print(f"{sensor}: {value:.3f}")

# Visualize weights
weights = np.linspace(0.1, 1.0, 10)
weights = weights / np.sum(weights)
plt.figure(figsize=(10, 4))
plt.bar(range(10), weights)
plt.title("Sensor Reading Weights")
plt.xlabel("Reading Number")
plt.ylabel("Weight")
plt.show()

## Input Validation Functions

Create functions to validate the data inputs:

In [11]:
def validate_taste_rating(value, name):
    """Validate taste rating is between 0 and 1."""
    try:
        value = float(value)
        if not (0 <= value <= 1):
            raise ValueError(f"{name} rating must be between 0 and 1")
        return value
    except ValueError as e:
        raise ValueError(f"Invalid {name} rating: {str(e)}")

def validate_sensor_reading(value, name):
    """Validate sensor reading is a positive float."""
    try:
        value = float(value)
        if value < 0:
            raise ValueError(f"{name} reading must be positive")
        return value
    except ValueError as e:
        raise ValueError(f"Invalid {name} reading: {str(e)}")

def validate_dilution(value):
    """Validate dilution percentage is between 0 and 100."""
    try:
        value = float(value)
        if not (0 <= value <= 100):
            raise ValueError("Dilution must be between 0 and 100")
        return value
    except ValueError as e:
        raise ValueError(f"Invalid dilution: {str(e)}")

def validate_effectiveness(value):
    """Validate effectiveness score is between 0 and 1."""
    try:
        value = float(value)
        if not (0 <= value <= 1):
            raise ValueError("Effectiveness must be between 0 and 1")
        return value
    except ValueError as e:
        raise ValueError(f"Invalid effectiveness: {str(e)}")

# Test validation functions
test_values = {
    'taste_rating': 0.7,
    'sensor_reading': 0.56,
    'dilution': 50,
    'effectiveness': 0.85
}

print("Validation test results:")
print(f"Taste rating: {validate_taste_rating(test_values['taste_rating'], 'sweet')}")
print(f"Sensor reading: {validate_sensor_reading(test_values['sensor_reading'], 'as7263_r')}")
print(f"Dilution: {validate_dilution(test_values['dilution'])}")
print(f"Effectiveness: {validate_effectiveness(test_values['effectiveness'])}")

Validation test results:
Taste rating: 0.7
Sensor reading: 0.56
Dilution: 50.0
Effectiveness: 0.85


## Taste Scale Definition

Define standardized scales for taste ratings with clear descriptions:

In [12]:
TASTE_SCALES = {
    'sweet': {
        0.0: "Not sweet at all",
        0.2: "Slightly sweet",
        0.4: "Moderately sweet",
        0.6: "Distinctly sweet",
        0.8: "Very sweet",
        1.0: "Extremely sweet"
    },
    'sour': {
        0.0: "Not sour at all",
        0.2: "Slightly sour",
        0.4: "Moderately sour",
        0.6: "Distinctly sour",
        0.8: "Very sour",
        1.0: "Extremely sour"
    },
    'salty': {
        0.0: "Not salty at all",
        0.2: "Slightly salty",
        0.4: "Moderately salty",
        0.6: "Distinctly salty",
        0.8: "Very salty",
        1.0: "Extremely salty"
    },
    'bitter': {
        0.0: "Not bitter at all",
        0.2: "Slightly bitter",
        0.4: "Moderately bitter",
        0.6: "Distinctly bitter",
        0.8: "Very bitter",
        1.0: "Extremely bitter"
    },
    'pungent': {
        0.0: "Not pungent at all",
        0.2: "Slightly pungent",
        0.4: "Moderately pungent",
        0.6: "Distinctly pungent",
        0.8: "Very pungent",
        1.0: "Extremely pungent"
    },
    'astringent': {
        0.0: "Not astringent at all",
        0.2: "Slightly astringent",
        0.4: "Moderately astringent",
        0.6: "Distinctly astringent",
        0.8: "Very astringent",
        1.0: "Extremely astringent"
    }
}

# Function to get description for a taste value
def get_taste_description(taste, value):
    """Get the closest description for a taste value."""
    value = float(value)
    closest_key = min(TASTE_SCALES[taste].keys(), key=lambda k: abs(k - value))
    return TASTE_SCALES[taste][closest_key]

# Test taste descriptions
test_value = 0.7
print(f"Test value: {test_value}")
for taste in TASTE_COLUMNS:
    desc = get_taste_description(taste, test_value)
    print(f"{taste.capitalize()}: {desc}")

Test value: 0.7
Sweet: Distinctly sweet
Sour: Distinctly sour
Salty: Distinctly salty
Bitter: Distinctly bitter
Pungent: Distinctly pungent
Astringent: Distinctly astringent


## Data Collection Form

Create an interactive form for collecting taste profile data:

In [ ]:
class TasteProfileForm:
    def __init__(self):
        # Sample info
        self.sample_id = widgets.Text(description='Sample ID:')
        self.factory_name = widgets.Text(description='Factory:')
        self.medicine_name = widgets.Text(description='Medicine:')
        
        # Sensor readings (10 samples)
        self.num_samples = 10
        self.current_sample = 0
        self.readings = []
        
        # Create sensor reading inputs
        self.alcohol_ppm = widgets.FloatText(description='Alcohol PPM:')
        self.as7263_r = widgets.FloatText(description='R Channel:')
        self.as7263_s = widgets.FloatText(description='S Channel:')
        self.as7263_t = widgets.FloatText(description='T Channel:')
        self.as7263_u = widgets.FloatText(description='U Channel:')
        self.as7263_v = widgets.FloatText(description='V Channel:')
        self.as7263_w = widgets.FloatText(description='W Channel:')
        
        # Taste ratings
        self.taste_sliders = {}
        for taste in TASTE_COLUMNS:
            self.taste_sliders[taste] = widgets.FloatSlider(
                value=0,
                min=0,
                max=1,
                step=0.1,
                description=f'{taste.capitalize()}:',
                continuous_update=True
            )
            
        # Quality and Dilution
        self.quality = widgets.FloatSlider(
            value=0.5,
            min=0,
            max=1,
            step=0.1,
            description='Quality:'
        )
        self.dilution = widgets.FloatSlider(
            value=100,
            min=0,
            max=100,
            step=10,
            description='Dilution %:'
        )
        
        # Add/Submit buttons
        self.add_reading = widgets.Button(description='Add Reading')
        self.add_reading.on_click(self.on_add_reading)
        self.submit = widgets.Button(description='Submit')
        self.submit.on_click(self.on_submit)
        
        # Status display
        self.status = widgets.HTML("")
        
        # Display form
        self.display_form()
        
    def display_form(self):
        """Display the data collection form."""
        display(widgets.VBox([
            widgets.HTML("<h3>Sample Information</h3>"),
            self.sample_id,
            self.factory_name,
            self.medicine_name,
            widgets.HTML("<h3>Sensor Readings</h3>"),
            self.alcohol_ppm,
            self.as7263_r,
            self.as7263_s,
            self.as7263_t,
            self.as7263_u,
            self.as7263_v,
            self.as7263_w,
            self.add_reading,
            self.status,
            widgets.HTML("<h3>Taste Profile</h3>"),
            *self.taste_sliders.values(),
            widgets.HTML("<h3>Quality and Dilution</h3>"),
            self.quality,
            self.dilution,
            self.submit
        ]))
        
    def on_add_reading(self, b):
        """Handle adding a sensor reading."""
        try:
            reading = {
                'alcohol_ppm': validate_sensor_reading(self.alcohol_ppm.value, 'alcohol_ppm'),
                'as7263_r': validate_sensor_reading(self.as7263_r.value, 'R'),
                'as7263_s': validate_sensor_reading(self.as7263_s.value, 'S'),
                'as7263_t': validate_sensor_reading(self.as7263_t.value, 'T'),
                'as7263_u': validate_sensor_reading(self.as7263_u.value, 'U'),
                'as7263_v': validate_sensor_reading(self.as7263_v.value, 'V'),
                'as7263_w': validate_sensor_reading(self.as7263_w.value, 'W')
            }
            
            self.readings.append(reading)
            self.current_sample += 1
            
            if self.current_sample < self.num_samples:
                self.status.value = f"Reading {self.current_sample} of {self.num_samples} recorded"
            else:
                self.status.value = f"All {self.num_samples} readings collected. Ready to submit."
                
        except Exception as e:
            self.status.value = f"<span style='color: red'>Error adding reading: {str(e)}</span>"
    
    def get_data(self):
        """Collect all form data into a dictionary."""
        if len(self.readings) < self.num_samples:
            raise ValueError(f"Need {self.num_samples} readings. Only have {len(self.readings)}")
        
        # Calculate weighted averages
        sensor_data = weighted_average_readings(self.readings)
        
        data = {
            'sample_id': self.sample_id.value,
            'timestamp': datetime.now().isoformat(),
            'factory_name': self.factory_name.value,
            'medicine_name': self.medicine_name.value,
            'quality': validate_taste_rating(self.quality.value, 'quality'),
            'dilution': validate_dilution(self.dilution.value),
            **sensor_data  # Add weighted sensor readings
        }
        
        # Add taste ratings
        for taste in TASTE_COLUMNS:
            data[f'taste_{taste}'] = validate_taste_rating(
                self.taste_sliders[taste].value, 
                taste
            )
            
        return data
    
    def on_submit(self, b):
        """Handle form submission."""
        try:
            # Validate and collect data
            data = self.get_data()
            
            # Add to DataFrame
            global df
            df = pd.concat([df, pd.DataFrame([data])], ignore_index=True)
            
            print("Data collected successfully!")
            print("\nTaste Profile Summary:")
            for taste in TASTE_COLUMNS:
                value = data[f'taste_{taste}']
                desc = get_taste_description(taste, value)
                print(f"{taste.capitalize()}: {value:.1f} - {desc}")
            
            # Reset readings for next sample
            self.readings = []
            self.current_sample = 0
            self.status.value = "Ready for next sample"
                
        except Exception as e:
            print(f"Error collecting data: {str(e)}")

# Create and display form
form = TasteProfileForm()

## Data Export Functions

Functions to export collected data in various formats:

In [6]:
def export_to_csv(df, filepath):
    """Export data to CSV file."""
    try:
        df.to_csv(filepath, index=False)
        print(f"Data exported to {filepath}")
    except Exception as e:
        print(f"Error exporting to CSV: {str(e)}")

def export_to_json(df, filepath):
    """Export data to JSON file."""
    try:
        df.to_json(filepath, orient='records', indent=2)
        print(f"Data exported to {filepath}")
    except Exception as e:
        print(f"Error exporting to JSON: {str(e)}")

# Example export (uncomment to use)
# export_to_csv(df, 'data/taste_profiles.csv')
# export_to_json(df, 'data/taste_profiles.json')

## Sample Data Generation

Generate sample data to demonstrate the structure and validation:

In [13]:
# Generate sample data
sample_medicines = ['Ashwagandha', 'Turmeric', 'Tulsi', 'Neem']
sample_data = []

for i in range(5):
    sample = {
        'sample_id': f'TEST_{i+1}',
        'timestamp': datetime.now().isoformat(),
        'medicine_name': np.random.choice(sample_medicines),
        'dilution': np.random.choice([25, 50, 75, 100]),
        'temperature': round(np.random.uniform(20, 30), 1),
        'as7263_r': round(np.random.uniform(0.3, 0.7), 2),
        'as7263_s': round(np.random.uniform(0.3, 0.7), 2),
        'as7263_t': round(np.random.uniform(0.3, 0.7), 2),
        'as7263_u': round(np.random.uniform(0.3, 0.7), 2),
        'as7263_v': round(np.random.uniform(0.3, 0.7), 2),
        'as7263_w': round(np.random.uniform(0.3, 0.7), 2)
    }
    
    # Generate taste profiles
    for taste in TASTE_COLUMNS:
        sample[taste] = round(np.random.uniform(0, 1), 1)
    
    # Add effectiveness
    sample['effectiveness'] = round(np.random.uniform(0.5, 1.0), 2)
    
    sample_data.append(sample)

# Create sample DataFrame
sample_df = pd.DataFrame(sample_data)
print("Sample data generated:")
display(sample_df)

Sample data generated:


,sample_id,timestamp,medicine_name,dilution,temperature,as7263_r,as7263_s,as7263_t,as7263_u,as7263_v,as7263_w,sweet,sour,salty,bitter,pungent,astringent,effectiveness
0,TEST_1,2025-09-17T12:32:44.473416,Turmeric,50,23.3,0.40,0.43,0.35,0.33,0.66,0.37,0.5,0.1,0.7,1.0,0.5,0.4,0.92
1,TEST_2,2025-09-17T12:32:44.488136,Tulsi,100,29.6,0.62,0.68,0.43,0.32,0.32,0.35,1.0,0.7,0.1,0.3,0.7,0.4,0.80
2,TEST_3,2025-09-17T12:32:44.490351,Neem,25,25.7,0.61,0.66,0.40,0.53,0.37,0.41,0.2,0.6,0.3,0.1,0.5,0.2,0.79
3,TEST_4,2025-09-17T12:32:44.490921,Turmeric,75,20.6,0.50,0.68,0.41,0.62,0.66,0.70,0.9,1.0,0.9,0.7,0.1,0.8,0.78
4,TEST_5,2025-09-17T12:32:44.490921,Neem,25,26.8,0.56,0.50,0.45,0.44,0.60,0.45,0.1,0.3,0.1,0.7,0.6,0.9,0.62


## Training Taste Profile Models

Train models to predict taste profiles from sensor readings:

In [ ]:
# Define model training function with confidence calculation
def train_taste_model(X, y, model_name):
    """Train a model with variance-based confidence calculation."""
    # Create and train model
    model = RandomForestRegressor(
        n_estimators=100,
        random_state=42,
        n_jobs=-1  # Use all CPU cores
    )
    model.fit(X, y)
    
    # Calculate predictions and confidence
    y_pred = model.predict(X)
    
    # Calculate tree variance for confidence
    tree_predictions = np.array([tree.predict(X) for tree in model.estimators_])
    variance = np.var(tree_predictions, axis=0)
    confidence = 1 / (1 + variance)
    confidence = np.clip(confidence, 0.1, 0.99)  # Clip confidence between 0.1 and 0.99
    
    # Print model performance
    r2 = r2_score(y, y_pred)
    mse = mean_squared_error(y, y_pred)
    print(f"\nModel: {model_name}")
    print(f"R² Score: {r2:.3f}")
    print(f"MSE: {mse:.3f}")
    print(f"Average Confidence: {np.mean(confidence):.3f}")
    
    return model, confidence

# Prepare features
X = sample_df[SENSOR_COLUMNS].values

# Train models for each taste
taste_models = {}
confidence_scores = {}

for taste in TASTE_COLUMNS:
    print(f"\nTraining model for {taste}...")
    y = sample_df[f'taste_{taste}'].values
    model, confidence = train_taste_model(X, y, f"{taste}_model")
    
    # Store model and confidence
    taste_models[taste] = model
    confidence_scores[taste] = confidence
    
    # Save model
    model_path = Path('../models')
    model_path.mkdir(parents=True, exist_ok=True)
    joblib.dump(model, model_path / f"taste_{taste}_model.pkl")
    print(f"Model saved to {model_path}/taste_{taste}_model.pkl")

# Train quality and dilution models
quality_model, quality_confidence = train_taste_model(X, sample_df['quality'].values, "quality_model")
dilution_model, dilution_confidence = train_taste_model(X, sample_df['dilution'].values, "dilution_model")

# Save quality and dilution models
joblib.dump(quality_model, model_path / "quality_model.pkl")
joblib.dump(dilution_model, model_path / "dilution_model.pkl")

print("\nAll models trained and saved successfully!")

# Create feature importance visualization
plt.figure(figsize=(12, 6))
importances = pd.DataFrame(
    index=SENSOR_COLUMNS,
    columns=TASTE_COLUMNS + ['quality', 'dilution']
)

for taste in TASTE_COLUMNS:
    importances[taste] = taste_models[taste].feature_importances_

importances['quality'] = quality_model.feature_importances_
importances['dilution'] = dilution_model.feature_importances_

sns.heatmap(importances, annot=True, cmap='YlOrRd', fmt='.3f')
plt.title('Feature Importance Across Models')
plt.tight_layout()
plt.show()


Training model for sweet...
Model saved to ../models/taste_sweet_model.pkl

Training model for sour...
Model saved to ../models/taste_sweet_model.pkl

Training model for sour...
Model saved to ../models/taste_sour_model.pkl

Training model for salty...
Model saved to ../models/taste_sour_model.pkl

Training model for salty...
Model saved to ../models/taste_salty_model.pkl

Training model for bitter...
Model saved to ../models/taste_salty_model.pkl

Training model for bitter...
Model saved to ../models/taste_bitter_model.pkl

Training model for pungent...
Model saved to ../models/taste_bitter_model.pkl

Training model for pungent...
Model saved to ../models/taste_pungent_model.pkl

Training model for astringent...
Model saved to ../models/taste_astringent_model.pkl

All taste profile models trained and saved!
Model saved to ../models/taste_pungent_model.pkl

Training model for astringent...
Model saved to ../models/taste_astringent_model.pkl

All taste profile models trained and saved!

: 